## Scheme_creation_script for MIP (Fresh/Takeover/Renewal) and SPS

IF FLEXIPF  is present THEN enter PF_MIN AND PF_MAX

IF FIXED PF  THEN only  enter PF_MIN 

In [19]:
# !pip install -qqqqq pandas jinja2 numpy google-auth openpyxl gspread 

In [20]:
import pandas as pd
import json,numpy as np
import jinja2   
from jinja2 import Template
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from decimal import Decimal, getcontext, ROUND_HALF_UP
from typing import Optional
import gspread
from google.oauth2.service_account import Credentials

MIP 24M CALCULATOR


New Calculator

In [21]:
# High precision for internal math
getcontext().prec = 28

def mip_24_months(
    cx_ltv: int,
    monthly_roi,
    secure_ltv: int,
    pf_min=None,
    pf_max=None,
    fc=None,
    ts_min: int = None,
    ts_max: int = None,
):
    """MIP 24-month calculator using Decimal (exact math)"""

    # ---- Convert inputs to Decimal ----
    cx_ltv = Decimal(cx_ltv)
    secure_ltv = Decimal(secure_ltv)
    monthly_roi = Decimal(str(monthly_roi))

    pf_min = Decimal(str(pf_min)) if pf_min is not None else None
    pf_max = Decimal(str(pf_max)) if pf_max is not None else None
    fc = Decimal(str(fc)) if fc is not None else None

    # ---- Constants ----
    tenure = Decimal("24")
    secure_irr = Decimal("14.0")
    secure_pf = Decimal("0.00")
    TWOPLACES = Decimal("0.01")

    # ---- LTV calculation ----
    unsecure_ltv = cx_ltv - secure_ltv

    # ---- Interest calculation ----
    annual_irr = monthly_roi * Decimal("12")

    # ---- Unsecured IRR ----
    unsec_irr = (
        (cx_ltv * annual_irr) - (secure_ltv * secure_irr)
    ) / unsecure_ltv

    # ---- Flexi PF calculation ----
    min_percent_unsecure = Decimal("0.00")
    max_percent_unsecure = Decimal("0.00")

    if pf_min is not None:
        min_percent_unsecure = (
            (pf_min * cx_ltv) - (secure_pf * secure_ltv)
        ) / unsecure_ltv

    if pf_max is not None:
        max_percent_unsecure = (
            (pf_max * cx_ltv) - (secure_pf * secure_ltv)
        ) / unsecure_ltv

    # ---- FC calculation ----
    fc_charges = Decimal("0.00")
    if fc is not None and unsecure_ltv != 0:
        fc_charges = (
            (fc * cx_ltv) - (secure_pf * secure_ltv)
        ) / unsecure_ltv

    # ---- FINAL ROUNDING and output----
    return (
        annual_irr.quantize(TWOPLACES, rounding=ROUND_HALF_UP),
        unsecure_ltv,
        unsec_irr.quantize(TWOPLACES, rounding=ROUND_HALF_UP),
        min_percent_unsecure.quantize(TWOPLACES, rounding=ROUND_HALF_UP),
        max_percent_unsecure.quantize(TWOPLACES, rounding=ROUND_HALF_UP),
        fc_charges.quantize(TWOPLACES, rounding=ROUND_HALF_UP),
        secure_irr.quantize(TWOPLACES, rounding=ROUND_HALF_UP),
    )


In [22]:
x=list(mip_24_months(cx_ltv=84,                  #Trial run of the  MIP calculator
              monthly_roi=1.59, 
            secure_ltv=71,      
            pf_min=0.7,
            ts_min=30000,
            ts_max=299999))
print("the annual_irr is ",x[0])
print("the unsecure_ltv is ",x[1])
print("the unsecure_irr is ",x[2])
print("the min_percent_unsecure is ",x[3])
print("the max_percent_unsecure is ",x[4])
print("the fc_charges is ",x[5])
print("the secure_irr is ",x[6])


the annual_irr is  19.08
the unsecure_ltv is  13
the unsecure_irr is  46.82
the min_percent_unsecure is  4.52
the max_percent_unsecure is  0.00
the fc_charges is  0.00
the secure_irr is  14.00


Load the template_path


In [23]:
# def load_templates(template_path: str):
#     df = pd.read_csv(template_path)
#     df=df.dropna(how='all',axis=0)
#     return df.to_dict(orient="records")

In [24]:
def load_templates(
    service_account_json: str,
    spreadsheet_id: str,
    worksheet_name: str,
):
    """
    Reads a Google Sheet tab using a service account and
    returns list[dict] (same output as your CSV version)
    """

    scopes = [
        "https://www.googleapis.com/auth/spreadsheets.readonly"
    ]

    creds = Credentials.from_service_account_file(
        service_account_json,
        scopes=scopes
    )

    client = gspread.authorize(creds)

    worksheet = client.open_by_key(spreadsheet_id).worksheet(worksheet_name)

    records = worksheet.get_all_records()
    df = pd.DataFrame(records)

    df = df.dropna(how="all", axis=0)

    return df.to_dict(orient="records")


Load the sample.csv


In [25]:
def load_sample_input(file_path: str):
    """
    Loads the sample.csv exactly as it is without changing header casing.
    """
    # 1. Read the CSV directly
    column_dtypes = {
        "template_type": str,
        'cx_ltv':int,
        'monthly_roi': float,        
        'city_id': str,
        'secure_ltv':int,
        'pf_min':float,
        'pf_max':float,
        'upi_mandate':str,
        'fc':float,
        'ts_min':int,
        'ts_max':int,
        'tag': str}
    df = pd.read_csv(file_path,dtype=column_dtypes)    
    df.columns = [str(c).replace("\ufeff", "").strip() for c in df.columns]
    df = df.dropna(how="all", axis=0)
    input_rows = df.to_dict(orient="records")
    return df.to_dict(orient="records")

Cusstomer ltv integration


In [26]:
def get_ltv_code(ltv: int) -> str:
    mapping = {
        70: "s0", 71: "s1", 72: "s2", 73: "s3", 74: "s4", 
        75: "s5", 76: "s6", 77: "s7", 78: "s8", 79: "s9",
        80: "e0", 81: "e1", 82: "e2", 83: "e3", 84: "e4", 
        85: "e5", 86: "e6", 87: "e7", 88: "e8", 89: "e9",
        90: "n0"
    }
    
    if ltv in mapping:
        return mapping[ltv]
    
    raise ValueError(f"No LTV code defined for ltv={ltv}")

ticket size in refName


In [27]:
def get_slab_label(ts_min: int, ts_max: int) -> str:
    if ts_max < 300000:
        return "<3L"

    if 300000 <= ts_min and ts_max <= 599999:
        return "3-6L"

    if 600000 <= ts_min and ts_max <= 1199999:
        return "6-12L"

    if ts_max >= 1200000:
        return ">12L"

    raise ValueError(f"Unknown slab")

UPI TAG

In [28]:
def upi_enabled(upi_mandate:str):
    if upi_mandate == "True":
        return "True"
    else:
        return "False"

End tag if any  at refName

In [29]:
#adding any tag at the end of refname
def tagged(tag):
    if pd.isna(tag) or tag is None:
        return ""
    return str(tag).upper()

Highlighting_logic

In [30]:
def highlight_columns(excel_path:str,
                      cols_to_hightlight:list[str],
                      color:str= "FFFF00"):
    wb=load_workbook(excel_path)
    ws=wb.active

    fill=PatternFill(start_color=color,end_color=color,fill_type="solid")

    header_to_col = {
        cell.value: idx + 1
        for idx, cell in enumerate(ws[1])
        if cell.value
    }

    for col_name in cols_to_hightlight:
        if col_name not in header_to_col:
            print(f"⚠️ Column not found, skipping: {col_name}")
            continue

        col_idx = header_to_col[col_name]
        for row in range(1, ws.max_row + 1):
            ws.cell(row=1, column=col_idx).fill = fill

    wb.save(excel_path)

In [31]:
HIGHLIGHT_COLS = ['refName','schemeFlags','SchemeMin','SchemeMax',   #all the columns to be highlighted and checked
                  'CityIds','chargeText','refno','customerLtv',
                  'OverallInterestCalculation','bs1-ltv','bs1-calculation','bs2-legalName',
                  'bs2-calculation','bs2-charge-2','bs2-charge-3']

Scheme validator

In [32]:
def scheme_validator(
    ts_min: int,
    ts_max: int,
    cx_ltv: int,
    secure_ltv: int,
    monthly_roi,
    annual_irr,
    unsec_irr,
    pf_min=None,
    pf_max=None,
    fc=None,
):
    errors = []
    TWOPLACES = Decimal("0.01")


    # ---- Convert to Decimal ----
    cx_ltv = Decimal(cx_ltv)
    secure_ltv = Decimal(secure_ltv)
    monthly_roi = Decimal(str(monthly_roi))
    annual_irr = Decimal(str(annual_irr))
    unsec_irr = Decimal(str(unsec_irr))

    pf_min = Decimal(str(pf_min)) if pf_min is not None else None
    pf_max = Decimal(str(pf_max)) if pf_max is not None else None
    fc = Decimal(str(fc)) if fc is not None else None

    # Constants
    secure_irr = Decimal("14.0")
    secure_pf = Decimal("0.00")

    #ticket size
    if ts_min is None or ts_max is None or ts_min >= ts_max:
        errors.append("Ticket Size range is invalid")

    # LTV checks
    
    if cx_ltv <= 0:
        errors.append("CX LTV must be greater than 0")

    if secure_ltv < 0 or secure_ltv > cx_ltv:
        errors.append("Secure LTV must be between 0 and CX LTV")

    unsecure_ltv = cx_ltv - secure_ltv
    if unsecure_ltv <= 0:
        errors.append("Unsecure LTV must be greater than 0")

    # ------------------------
    # Reverse check 1: annual ↔ monthly
    # ------------------------
    derived_monthly = (annual_irr / Decimal("12")).quantize(TWOPLACES)
    if derived_monthly != monthly_roi:
        errors.append(
            f"Monthly ROI reverse check failed "
            f"(expected {derived_monthly}, got {monthly_roi})"
        )

    # ------------------------
    # Reverse check 2: unsecure IRR ↔ annual
    # ------------------------
    derived_annual = (
        (unsec_irr * unsecure_ltv) + (secure_ltv * secure_irr)
    ) / cx_ltv
    derived_annual = derived_annual.quantize(TWOPLACES)
    if derived_annual != annual_irr:
        errors.append(
            f"Unsecure IRR reverse check failed "
            f"(expected {derived_annual}, got {annual_irr})"
        )

    # ------------------------
    # Reverse check 3: PF MIN
    # ------------------------
    if pf_min is not None:
        derived_pf_min = (
            (pf_min * cx_ltv) - (secure_pf * secure_ltv)
        ) / unsecure_ltv
        
        reconstructed_pf_min = (
            (derived_pf_min * unsecure_ltv) + (secure_pf * secure_ltv)
        ) / cx_ltv
        reconstructed_pf_min = reconstructed_pf_min.quantize(TWOPLACES)
        if reconstructed_pf_min != pf_min:
            errors.append("PF Min reverse check failed")

    # ------------------------
    # Reverse check 4: PF MAX
    # ------------------------
    if pf_max is not None and pf_max!= Decimal("0.00"):
        derived_pf_max = (
            (pf_max * cx_ltv) - (secure_pf * secure_ltv)
        ) / unsecure_ltv

        reconstructed_pf_max = (
            (derived_pf_max * unsecure_ltv) + (secure_pf * secure_ltv)
        ) / cx_ltv
        reconstructed_pf_max = reconstructed_pf_max.quantize(TWOPLACES)
        if reconstructed_pf_max != pf_max:
            errors.append("PF Max reverse check failed")

    # ------------------------
    # Reverse check 5: FC
    # ------------------------
    if fc is not None and fc != Decimal("0.00"):
        derived_fc = (
            (fc * cx_ltv) - (secure_pf * secure_ltv)
        ) / unsecure_ltv

        reconstructed_fc = (
            (derived_fc * unsecure_ltv) + (secure_pf * secure_ltv)
        ) / cx_ltv
        reconstructed_fc = reconstructed_fc.quantize(TWOPLACES)
        if reconstructed_fc != fc:
            errors.append("FC reverse check failed")

    #final
    if errors:
        return {
            "status": "INVALID",
            "errors": errors
        }

    return {
        "status": "VALID"
    }

Main function to make the schemes


In [34]:
def generate_schemes(input_path,output_path):
    print("✅ Initializing Scheme Generation")

    # 1. Loading Data
    print(f"✅ Loading input file: {input_path}")
    input_rows = load_sample_input(input_path)

    print(f"✅ Loading template file fromt the google sheet")

    ##service acccount json need to be changed

    SERVICE_ACCOUNT_JSON = r"C:\Users\Ingit.Paul.in\Downloads\schemes-486410-773d0189a4ce.json"
    SPREADSHEET_ID = "18QsZeCu0vEET6p-wnxhAljWG8JpQOUTqwVhmJSesN00"
    WORKSHEET_NAME = "MIP_TEMPLATE"  

    template_rows = load_templates(SERVICE_ACCOUNT_JSON,SPREADSHEET_ID,WORKSHEET_NAME)
    
    final_schemes = []
    total_rows = len(input_rows)
    print(f"✅ Data loaded. Found {total_rows} rows to process.")
    print("-" * 50)

    for i, row in enumerate(input_rows, 1):
        # Progress indicator
        print(f"🔄 Processing Row [{i}/{total_rows}] | Type: {row.get('template_type')}...", end="\r")

        #Helper to safely get float values
         
        from decimal import Decimal

        def get_decimal_or_none(row, key):
            val = row.get(key)
            if val is None or str(val).strip().lower() in ("nan", ""):
                return None
            return Decimal(str(val))


        #Logic Mapping (LTV & Slab)
        current_ltv = int(row.get("cx_ltv"))
        ltv_code = get_ltv_code(current_ltv)
          
        secure_ltv = row.get("secure_ltv")
        monthly_roi = row.get("monthly_roi")

        
        ts_min = int(row.get("ts_min"))
        ts_max = int(row.get("ts_max"))
        slab_label = get_slab_label(ts_min, ts_max) 
        #UPI TAG
        upi_tag = upi_enabled(row.get("upi_mandate"))

        #city
        city_id = row.get("city_id")

        #adding tags at the end
        end_tags=tagged(row.get("tag"))
    
        #Run Calculation (MIP24M)
        (annual_irr, unsecure_ltv, unsec_irr, min_percent_unsecure, max_percent_unsecure, 
        fc_charges, secure_irr)= mip_24_months(
            cx_ltv=get_decimal_or_none(row, "cx_ltv"),
            monthly_roi=get_decimal_or_none(row, "monthly_roi"),
            secure_ltv=get_decimal_or_none(row, "secure_ltv"),
            pf_min=get_decimal_or_none(row, "pf_min"),
            pf_max=get_decimal_or_none(row, "pf_max"),
            fc=get_decimal_or_none(row, "fc")
        )

        #validation check
        validation = scheme_validator(
            ts_min=row.get("ts_min"),
            ts_max=row.get("ts_max"),
            cx_ltv=get_decimal_or_none(row, "cx_ltv"),
            secure_ltv=get_decimal_or_none(row, "secure_ltv"),
            monthly_roi=get_decimal_or_none(row, "monthly_roi"),
            annual_irr=annual_irr,
            unsec_irr=unsec_irr,
            pf_min=get_decimal_or_none(row, "pf_min"),
            pf_max=get_decimal_or_none(row, "pf_max"),
            fc=get_decimal_or_none(row, "fc"),
        )
        if validation["status"] == "INVALID":
            error_msg = ", ".join(validation["errors"])
            print(f"\n❌ Skipping Row {i} | Error: {error_msg}")
            continue

        # 4. Build Context (Adding the new labels here)
        context = {
            **row,
            "end_tags": end_tags,
            "upi_tag": upi_tag,
            "ltv_code": ltv_code,      
            "slab_label": slab_label,  
            "annual_irr": annual_irr,
            "city_id":city_id,    
            "unsec_irr": unsec_irr,
            "min_percent_unsecure": min_percent_unsecure,
            "max_percent_unsecure": max_percent_unsecure,
            "fc_charges": fc_charges,
            "secure_irr": secure_irr
        }
        
        #cleaning the NaN values
        for k, v in context.items():
            if pd.isna(v):
                context[k] = ""
            elif isinstance(v, float):
                context[k] = f"{v:.2f}"
            else:
                context[k] = v

        # Matching & Rendering templates
        t_type = row.get("template_type")
        template = next((t for t in template_rows if t.get("template_type") == t_type), None)

        if template:
            rendered_scheme = {}
            for key, value in template.items():
                if isinstance(value, str) and "{{" in value:
                    rendered_scheme[key] = Template(value).render(context)
                else:
                    rendered_scheme[key] = value
            
            final_schemes.append(rendered_scheme)
        else:
            print(f"\n⚠️ Warning: No template match for type '{t_type}' at row {i}")

    print(f"\n\n✨ All rows processed!")
    print("-" * 50)

    # 6. Output to New File
    if final_schemes:
        print(f"✅Saving results to: {output_path}")
        df_output = pd.DataFrame(final_schemes)
        
        if "template_type" in df_output.columns:
            df_output = df_output.drop(columns=["template_type"])
            
        df_output.to_excel(output_path, index=False)
        print(f"🎊 Success! {len(final_schemes)} schemes generated successfully.")
        
        #Highlighting important columns      

        highlight_columns(excel_path=output_path,cols_to_hightlight=HIGHLIGHT_COLS)

        print("✅Check the Highlighted columns in the output Excel file.")
    else:
        print("❌Error: No schemes were generated. Please check your template_type values.")

if __name__ == "__main__":
    generate_schemes(
        input_path=r"C:\jinja_schemes\mipfed 24\sample.csv",         ## UPLOAD THE CSV FILE FROM HERE
        output_path=r"C:\jinja_schemes\output9.xlsx"              ## THIS IS WHERE YOU WANT TO SAVE THE OUTPUT FILE
    )

✅ Initializing Scheme Generation
✅ Loading input file: C:\jinja_schemes\mipfed 24\sample.csv
✅ Loading template file fromt the google sheet
✅ Data loaded. Found 4 rows to process.
--------------------------------------------------
🔄 Processing Row [4/4] | Type: MIP_Renewal...

✨ All rows processed!
--------------------------------------------------
✅Saving results to: C:\jinja_schemes\output9.xlsx
🎊 Success! 4 schemes generated successfully.
✅Check the Highlighted columns in the output Excel file.


# End of the main loop
